# Problem Statement 1 :

Scenario:By analyzing text data, we can find meaningful insights from non-numeric data that can help usachieve our objective.With the help ofNLP and its concepts,we can do it.Twitterisone of the biggest platformsthatpeopleuse to write their messages, express their feelingsabouta particular topic,and share knowledgein the form oftext. By analyzingtext data,we can make good decisionsfor different use cases likejudging the sentiment of the human tweets, and anyproduct review/comments can tell us the performance of a product in the market.NLP allowsus to study and understandthe colinearity of the data.Sowecanpredict our objective. Objective: Use Python libraries such as Pandasfor data operations, Seabornand Matplotlibfor data visualization and EDA tasks,NLTKtoextract and analyze the information,Sklearnfor model building and performance visualization, to predict our different categories of people’s mindsets.Dataset description:The data contain information about manyTweets in the form oftext and their types,as mentionedbelow.Tweets: Data is in the form ofa sentence written by individuals.category:  Numeric(0: Neutral, -1: Negative, 1: Positive)(It is our dependent variable)The following tasks are to be performed:•Read the Data from the Given excel file.•Change our dependent variable to categorical. (0 to “Neutral,”-1 to “Negative”, 1 to “Positive”)•Do Missing value analysisand drop all null/missing values•Do text cleaning. (remove every symbol except alphanumeric, transform all words to lower case, and remove punctuationand stopwords )•Create a new column and find the length of each sentence (how many words they contain)•Split data into dependent(X) and independent(y) dataframe•Do operations on text data Hints:Do one-hot encoding for each sentence(use TensorFlow)oAdd padding from the front side (use Tensorflow)oBuild anLSTM model and compile it(describe features, input length, vocabulary size, information drop-out layer, activation function for output, )oDo dummy variable creation for the dependent variableosplit the data into tests and train •Train new model•Normalize the prediction as same as the originaldata(prediction might be in decimal, so whoever is nearest to 1 is predicted as yes and set other as 0)•Measure performance metricsand accuracy•print Classification report

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.preprocessing import normalize
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [2]:
# Task 1: Read the Data from the Given excel file.
file_path = r'C:\Users\King\Desktop\DSML Internship\Assignments\Mini Project (Deep Learning)\Dataset\Twitter_Data.csv'
df = pd.read_csv(file_path)
df

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
162975,why these 456 crores paid neerav modi not reco...,-1.0
162976,dear rss terrorist payal gawar what about modi...,-1.0
162977,did you cover her interaction forum where she ...,0.0
162978,there big project came into india modi dream p...,0.0


In [3]:
# Task 2: Change our dependent variable to categorical. (0 to “Neutral,”-1 to “Negative”, 1 to “Positive”)
df['category'] = df['category'].map({0: 'Neutral', -1: 'Negative', 1: 'Positive'})
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,Negative
1,talk all the nonsense and continue all the dra...,Neutral
2,what did just say vote for modi welcome bjp t...,Positive
3,asking his supporters prefix chowkidar their n...,Positive
4,answer who among these the most powerful world...,Positive


In [4]:
# Task 3: Do Missing value analysis and drop all null/missing values
df.dropna(inplace=True)

In [5]:
df.isnull().sum()

clean_text    0
category      0
dtype: int64

In [6]:
# Task 4: Do text cleaning. (remove every symbol except alphanumeric, transform all words to lower case, and remove punctuationand stopwords )
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = ''.join([char.lower() for char in text if char.isalnum() or char.isspace()])
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['clean_text'] = df['clean_text'].apply(clean_text)

In [7]:
# Task 5: Create a new column and find the length of each sentence (how many words they contain)
df['sentence_length'] = df['clean_text'].apply(lambda x: len(x.split()))
df.head()

,clean_text,category,sentence_length
0,modi promised minimum government maximum gover...,Negative,21
1,talk nonsense continue drama vote modi,Neutral,6
2,say vote modi welcome bjp told rahul main camp...,Positive,13
3,asking supporters prefix chowkidar names modi ...,Positive,19
4,answer among powerful world leader today trump...,Positive,10


In [8]:
# Task 6: Split data into dependent(X) and independent(y) dataframe
X = df['clean_text']
y = df['category']

In [9]:
""" Task 7: Do operationson text data 
    Hints:Do one-hot encoding for each sentence(use TensorFlow)
      Add padding from the front side (use Tensorflow)
      Build an LSTM model and compile it(describe features, input length, vocabulary size, information drop-out layer, activation function for output, )
      Do dummy variable creation for the dependent variableosplit the data into tests and train 
      split the data into tests and train
"""

# Encode the Dependent Variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Determine the maximum sentence length
max_len = max(df['sentence_length'])

# One-hot encoding and padding for training set
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, padding='pre', maxlen=max_len) 

# One-hot encoding and padding for testing set
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, padding='pre', maxlen=max_len) 

# Building the  LSTM Model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=X_train_padded.shape[1]))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


C:\Users\King\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\King\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [10]:
# Task 8: Train the Model
model.fit(X_train_padded, y_train, epochs=3, batch_size=64, validation_data=(X_test_padded, y_test))

Epoch 1/3


2038/2038 [==============================] - 198s 95ms/step - loss: -23.4193 - accuracy: 0.3385 - val_loss: -43.1525 - val_accuracy: 0.3395
Epoch 2/3
2038/2038 [==============================] - 195s 96ms/step - loss: -64.7963 - accuracy: 0.3386 - val_loss: -83.8302 - val_accuracy: 0.3395
Epoch 3/3
2038/2038 [==============================] - 195s 96ms/step - loss: -113.9350 - accuracy: 0.3480 - val_loss: -169.5452 - val_accuracy: 0.4065


In [12]:
# Task 9: Normalize the prediction as same as the original data(prediction might be in decimal, so whoever is nearest to 1 is predicted as yes and set other as 0)
predictions = model.predict(X_test_padded)
normalized_predictions = np.round(normalize(predictions.reshape(-1, 1)))
normalized_predictions

1019/1019 [==============================] - 9s 9ms/step


array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [13]:
# Task 10: Measure performance metrics and accuracy
classification_rep = classification_report(y_test, normalized_predictions, target_names=label_encoder.classes_)
accuracy = np.sum(normalized_predictions.flatten() == y_test) / len(y_test)

C:\Users\King\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\King\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\King\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
# Task 11: Print Classification report
print("Classification Report:\n", classification_rep)
print("Accuracy:", accuracy)

Classification Report:
               precision    recall  f1-score   support

    Negative       0.87      0.04      0.07      7152
     Neutral       0.34      1.00      0.51     11067
    Positive       0.00      0.00      0.00     14375

    accuracy                           0.35     32594
   macro avg       0.40      0.35      0.19     32594
weighted avg       0.31      0.35      0.19     32594

Accuracy: 0.34733386512855124
